In [1]:
from datasets import load_from_disk

train_dir = '/opt/ml/input/data/data/train_dataset'
datasets = load_from_disk(train_dir)

In [2]:
datasets

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [3]:
type(datasets)

datasets.dataset_dict.DatasetDict

In [4]:
datasets['train']

Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 3952
})

In [5]:
datasets['train']['answers'][:10]

[{'answer_start': [235], 'text': ['하원']},
 {'answer_start': [212], 'text': ['《경영의 실제》']},
 {'answer_start': [510], 'text': ['백성']},
 {'answer_start': [625], 'text': ['중국']},
 {'answer_start': [30], 'text': ['4개']},
 {'answer_start': [91], 'text': ['드래곤']},
 {'answer_start': [68], 'text': ['형양태수 왕식']},
 {'answer_start': [583], 'text': ['이탈리아군']},
 {'answer_start': [195], 'text': ['큰아들 유']},
 {'answer_start': [861], 'text': ['왕대마을']}]

In [6]:
len(datasets['validation'])

240

In [117]:
test_dir = '/opt/ml/input/data/data/test_dataset'

test_datasets = load_from_disk(test_dir)

In [118]:
len(test_datasets['validation'])

600

In [127]:
print(test_datasets['validation']['question'][44])

유로니우스를 살해한 용의자가 소속된 밴드의 국적은?


- dataset으로 얻을 수 있는 정보

- 제목, 지문내용, 질문, id, 답(시작지점, 단어 혹은 문장), 문서의 고유 id

- 지문의 길이, 정답의 위치, 

In [10]:
!pwd

/opt/ml/code


In [11]:
import json
import os

data_path = "../input/data/data/"
context_path = "wikipedia_documents.json"
with open(os.path.join(data_path, context_path), "r") as f:
    wiki = json.load(f)
    
contexts = list(dict.fromkeys([v['text'] for v in wiki.values()]))

In [12]:
ids = list(range(len(contexts)))

In [13]:
print(len(contexts))

56737


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

In [15]:
print(contexts[0])
print(tokenizer.tokenize(contexts[0]))

이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.

이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.

# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.
# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.

두 목록은 모두 가나다 순이다.

일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.
['이', '문서', '##는', '나', '##라', '목', '##록', '##이며', ',', '전', '세계', '206', '##개', '나', '##라의', '각', '현', '##황', '##과', '주', '##권', '승', '##인', '정', '##보를', '개', '##요', '형', '##태로', '나', '##열', '##하고', '있다', '.', '이', '목', '##록', '##은', '명', '##료', '##화를', '위해', '두', '부', '##분', '##으로', '나', '##뉘', '##어', '있다', '.', '#', '첫', '번째', '부', '##분', '##은', '바', '##티', '##칸', '시', '##국', '##과', '팔', '##레스', '##타', '##인을', '포', '##함', '##하여', '유', '##엔', '등', '국제', '기', '##구', '##에', '가', '##입', '##되어', '국제', '##적인', '승', '##인을', '널리', '받았다', '##고',

In [16]:
from konlpy.tag import Mecab

mecab = Mecab()

In [17]:
print(mecab.morphs(contexts[0]))

['이', '문서', '는', '나라', '목록', '이', '며', ',', '전', '세계', '206', '개', '나라', '의', '각', '현황', '과', '주권', '승인', '정보', '를', '개요', '형태', '로', '나열', '하', '고', '있', '다', '.', '이', '목록', '은', '명료', '화', '를', '위해', '두', '부분', '으로', '나뉘', '어', '있', '다', '.', '#', '첫', '번', '째', '부분', '은', '바티칸', '시국', '과', '팔레스타인', '을', '포함', '하', '여', '유엔', '등', '국제', '기구', '에', '가입', '되', '어', '국제', '적', '인', '승인', '을', '널리', '받', '았', '다고', '여기', '는', '195', '개', '나라', '를', '나열', '하', '고', '있', '다', '.', '#', '두', '번', '째', '부분', '은', '일부', '지역', '의', '주권', '을', '사실', '상', '(', '데', '팍토', ')', '행사', '하', '고', '있', '지만', ',', '아직', '국제', '적', '인', '승인', '을', '널리', '받', '지', '않', '았', '다고', '여기', '는', '11', '개', '나라', '를', '나열', '하', '고', '있', '다', '.', '두', '목록', '은', '모두', '가', '나다', '순', '이', '다', '.', '일부', '국가', '의', '경우', '국가', '로서', '의', '자격', '에', '논쟁', '의', '여부', '가', '있', '으며', ',', '이', '때문', '에', '이러', '한', '목록', '을', '엮', '는', '것', '은', '매우', '어렵', '고', '논란', '이', '생길', '수', '있', '는', '과정', '이', '다', 

In [18]:
tfidfv = TfidfVectorizer(tokenizer=mecab.morphs,
                        ngram_range = (1,2),
                        max_features=100000,
                        )

In [19]:
print(tfidfv)

TfidfVectorizer(max_features=100000, ngram_range=(1, 2),
                tokenizer=<bound method Mecab.morphs of <konlpy.tag._mecab.Mecab object at 0x7f33e939a150>>)


In [20]:
import pickle

pickle_name = f"sparse_embedding"    # sparse_embedding binary 이름
tfidfv_name = f"tfidv"               # tfidv의 이름
emd_path = os.path.join(data_path, pickle_name)  
tfidfv_path = os.path.join(data_path, tfidfv_name)
if os.path.isfile(emd_path) and os.path.isfile(tfidfv_path):
    with open(emd_path, "rb") as file:
        p_embedding = pickle.load(file)
    with open(tfidfv_path, "rb") as file:
        tfidfv = pickle.load(file)
    print("Embedding pickle load.")
else:
    print("Build passage embedding")
    p_embedding = tfidfv.fit_transform(contexts)
    print(p_embedding.shape)
    with open(emd_path, "wb") as file:
        pickle.dump(p_embedding, file)
    with open(tfidfv_path, "wb") as file:
        pickle.dump(tfidfv, file)
    print("Embedding pickle saved.")

Embedding pickle load.


In [21]:
print(p_embedding)

  (0, 62658)	0.06647210844153642
  (0, 44295)	0.06337436691336805
  (0, 56)	0.0682445067500245
  (0, 78969)	0.06808191239523119
  (0, 79701)	0.03364157929519915
  (0, 34427)	0.07138338544472347
  (0, 27854)	0.05211670885220209
  (0, 4972)	0.06967047131796292
  (0, 64358)	0.016268006913672292
  (0, 95103)	0.01655220520966225
  (0, 53156)	0.09136510760391128
  (0, 72420)	0.03110653607650303
  (0, 29980)	0.06984183111934582
  (0, 83192)	0.11817707485598054
  (0, 34433)	0.05892170582042486
  (0, 60881)	0.0469695070469034
  (0, 27058)	0.06062239079676399
  (0, 30493)	0.06558610466405276
  (0, 95053)	0.01408397390908476
  (0, 53008)	0.0566188710751967
  (0, 40410)	0.06346765686416947
  (0, 24814)	0.05295014551043531
  (0, 30403)	0.04956238780026324
  (0, 24628)	0.04558964463399757
  (0, 70915)	0.050172357624740066
  :	:
  (56736, 34366)	0.024836628041649152
  (56736, 20488)	0.046616709303327286
  (56736, 95546)	0.011263209967785735
  (56736, 58883)	0.01911913768348471
  (56736, 86007)	0.0157

In [22]:
#print(tfidfv.get_feature_names())

In [23]:
pip install rank_bm25

Note: you may need to restart the kernel to use updated packages.


In [24]:
from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]
print(tokenized_corpus)

bm25 = BM25Okapi(tokenized_corpus)
# <rank_bm25.BM25Okapi at 0x1047881d0>

[['Hello', 'there', 'good', 'man!'], ['It', 'is', 'quite', 'windy', 'in', 'London'], ['How', 'is', 'the', 'weather', 'today?']]


In [25]:
print(bm25)

In [26]:
column_names = datasets["train"].column_names

In [27]:
print(column_names)

['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__']


In [28]:
question_column_name = "qeustion" if "question" in column_names else column_names[0]

In [29]:
print(question_column_name)

qeustion


In [30]:
context_column_name = "context" if "context" in column_names else column_names[1]
answer_column_name = "answers" if "answers" in column_names else column_names[2]

In [31]:
tokenizer = AutoTokenizer.from_pretrained(
        "bert-base-multilingual-cased",
        use_fast=True,
    )

In [32]:
pad_on_right = tokenizer.padding_side == "right"

In [33]:
print(pad_on_right)

True


In [34]:
print(tokenizer.padding_side)

right


In [35]:
train_dataset = datasets["train"]

In [36]:
type(datasets)

datasets.dataset_dict.DatasetDict

In [37]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [38]:
max_seq_length = 384

In [39]:
max_seq_length = min(max_seq_length, tokenizer.model_max_length)  # max_seq_length에 둘 중 더 작은걸 저장한다

In [40]:
doc_stride = 128

In [41]:
pad_to_max_length = False

In [42]:
tokenized_examples = tokenizer(
            train_dataset['question' if pad_on_right else 'context'],            #첫 문장 설정
            train_dataset['context' if pad_on_right else 'question'],            #두번쨰 문장 설정
            truncation="only_second" if pad_on_right else "only_first",                         #오른쪽 padding이면 두번째에서 자르고 아니면 첫번째에서 자름
            max_length=max_seq_length,                              # max_length
            stride=doc_stride,                            # 겹치는 tokenizer의 길이
            return_overflowing_tokens=True,                         # 넘치는 token 반환
            return_offsets_mapping=True,                            
                # pretrainedTokenizerFast로부터 상속 받은 tokenize의 경우에만 사용 가능, char_start와 char_end를 return
            padding="max_length" if pad_to_max_length else False,     # padding을 max_length까지 채운다
        )

In [43]:
print(tokenized_examples[0])

Encoding(num_tokens=384, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [44]:
print(tokenized_examples[0].tokens)

['[CLS]', '대통령', '##을', '포함한', '미국의', '행', '##정', '##부', '견', '##제', '##권을', '갖', '##는', '국가', '기', '##관은', '?', '[SEP]', '미국', '상', '##의', '##원', '또는', '미국', '상', '##원', '(', 'United', 'States', 'Senate', ')', '은', '양', '##원', '##제', '##인', '미국', '의', '##회의', '상', '##원이', '##다', '.', '\\', 'n', '\\', 'n', '##미', '##국', '부', '##통', '##령', '##이', '상', '##원의', '##장이', '된다', '.', '각', '주', '##당', '2', '##명의', '상', '##원의', '##원이', '선', '##출', '##되어', '100', '##명의', '상', '##원의', '##원으로', '구', '##성되어', '있다', '.', '임', '##기는', '6', '##년', '##이며', ',', '2년', '##마다', '50', '##개', '##주', '중', '1', '/', '3', '##씩', '상', '##원의', '##원을', '새', '##로', '선', '##출', '##하여', '연', '##방', '##에', '보', '##낸', '##다', '.', '\\', 'n', '\\', 'n', '##미', '##국', '상', '##원은', '미국', '하', '##원', '##과', '##는', '다', '##르게', '미국', '대통령', '##을', '수', '##반', '##으로', '하는', '미국', '연', '##방', '행', '##정', '##부에', '각', '##종', '동', '##의를', '하는', '기', '##관', '##이다', '.', '하', '##원이', '세', '##금', '##과', '경', '##제', '##에', '대한', '

In [45]:
print(tokenized_examples[0].ids)

[101, 70672, 10622, 104431, 37650, 9966, 16605, 14646, 8880, 17730, 57952, 8854, 11018, 93222, 8932, 101958, 136, 102, 23545, 9414, 10459, 14279, 20625, 23545, 9414, 14279, 113, 10609, 10859, 22294, 114, 9632, 9543, 14279, 17730, 12030, 23545, 9637, 56356, 9414, 73295, 11903, 119, 165, 182, 165, 182, 22458, 20479, 9365, 43022, 44220, 10739, 9414, 74125, 55635, 22143, 119, 8844, 9689, 21928, 123, 45441, 9414, 74125, 73295, 9428, 52363, 16855, 10407, 45441, 9414, 74125, 78686, 8908, 103764, 11506, 119, 9644, 46216, 127, 10954, 33542, 117, 92360, 101814, 10462, 21789, 16323, 9694, 122, 120, 124, 119108, 9414, 74125, 51684, 9415, 11261, 9428, 52363, 13374, 9568, 42337, 10530, 9356, 75884, 11903, 119, 165, 182, 165, 182, 22458, 20479, 9414, 84467, 23545, 9952, 14279, 11882, 11018, 9056, 78131, 23545, 70672, 10622, 9460, 30134, 11467, 23969, 23545, 9568, 42337, 9966, 16605, 52961, 8844, 22200, 9095, 78505, 23969, 8932, 20595, 11925, 119, 9952, 73295, 9435, 40032, 11882, 8885, 17730, 10530, 1

In [46]:
print(tokenized_examples[0].type_ids)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [47]:
print(tokenized_examples[0].offsets)

[(0, 0), (0, 3), (3, 4), (5, 8), (9, 12), (13, 14), (14, 15), (15, 16), (17, 18), (18, 19), (19, 21), (22, 23), (23, 24), (25, 27), (28, 29), (29, 31), (31, 32), (0, 0), (0, 2), (3, 4), (4, 5), (5, 6), (7, 9), (10, 12), (13, 14), (14, 15), (15, 16), (16, 22), (23, 29), (30, 36), (36, 37), (37, 38), (39, 40), (40, 41), (41, 42), (42, 43), (44, 46), (47, 48), (48, 50), (51, 52), (52, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (63, 64), (64, 65), (65, 66), (66, 67), (68, 69), (69, 71), (71, 73), (74, 76), (76, 77), (78, 79), (80, 81), (81, 82), (83, 84), (84, 86), (87, 88), (88, 90), (90, 92), (93, 94), (94, 95), (95, 97), (98, 101), (101, 103), (104, 105), (105, 107), (107, 110), (111, 112), (112, 115), (116, 118), (118, 119), (120, 121), (121, 123), (124, 125), (125, 126), (126, 128), (128, 129), (130, 132), (132, 134), (135, 137), (137, 138), (138, 139), (140, 141), (142, 143), (143, 144), (144, 145), (145, 146), (147, 148), (148, 150), (150, 1

In [48]:
print(tokenized_examples[0].attention_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [49]:
print(tokenized_examples[0].special_tokens_mask)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [50]:
print(tokenized_examples[0].overflowing)

[Encoding(num_tokens=294, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]


In [51]:
tokenized_examples["overflow_to_sample_mapping"]

[0,
 0,
 1,
 1,
 2,
 2,
 3,
 3,
 4,
 4,
 5,
 5,
 6,
 6,
 7,
 7,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 13,
 13,
 14,
 14,
 15,
 15,
 16,
 16,
 16,
 17,
 17,
 18,
 18,
 19,
 19,
 20,
 20,
 21,
 22,
 22,
 22,
 23,
 23,
 23,
 24,
 24,
 24,
 25,
 25,
 26,
 27,
 27,
 28,
 28,
 29,
 29,
 29,
 29,
 30,
 30,
 30,
 31,
 31,
 32,
 32,
 33,
 33,
 34,
 34,
 35,
 35,
 35,
 35,
 35,
 36,
 36,
 37,
 37,
 37,
 37,
 37,
 38,
 38,
 39,
 39,
 40,
 40,
 40,
 40,
 41,
 41,
 41,
 42,
 42,
 43,
 43,
 44,
 44,
 44,
 45,
 45,
 45,
 46,
 46,
 46,
 47,
 47,
 48,
 48,
 49,
 50,
 50,
 51,
 51,
 51,
 51,
 51,
 52,
 52,
 53,
 54,
 54,
 55,
 55,
 56,
 56,
 57,
 57,
 57,
 58,
 58,
 58,
 59,
 59,
 59,
 60,
 60,
 61,
 61,
 61,
 62,
 62,
 63,
 64,
 64,
 64,
 65,
 65,
 65,
 66,
 67,
 67,
 68,
 68,
 68,
 69,
 69,
 70,
 70,
 70,
 70,
 71,
 71,
 72,
 72,
 72,
 73,
 73,
 74,
 74,
 75,
 75,
 76,
 76,
 76,
 77,
 77,
 78,
 78,
 79,
 79,
 80,
 80,
 81,
 81,
 81,
 81,
 81,
 82,
 82,
 82,
 83,
 83

In [52]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

In [53]:
print(sample_mapping)

[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 9, 9, 9, 10, 10, 11, 11, 11, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 26, 27, 27, 28, 28, 29, 29, 29, 29, 30, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 35, 35, 35, 36, 36, 37, 37, 37, 37, 37, 38, 38, 39, 39, 40, 40, 40, 40, 41, 41, 41, 42, 42, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 47, 47, 48, 48, 49, 50, 50, 51, 51, 51, 51, 51, 52, 52, 53, 54, 54, 55, 55, 56, 56, 57, 57, 57, 58, 58, 58, 59, 59, 59, 60, 60, 61, 61, 61, 62, 62, 63, 64, 64, 64, 65, 65, 65, 66, 67, 67, 68, 68, 68, 69, 69, 70, 70, 70, 70, 71, 71, 72, 72, 72, 73, 73, 74, 74, 75, 75, 76, 76, 76, 77, 77, 78, 78, 79, 79, 80, 80, 81, 81, 81, 81, 81, 82, 82, 82, 83, 83, 83, 84, 84, 85, 85, 86, 86, 87, 87, 88, 88, 89, 89, 89, 89, 90, 90, 91, 92, 93, 93, 93, 94, 94, 94, 95, 95, 96, 96, 97, 97, 97, 97, 98, 98, 99, 99, 100, 100, 100, 101, 101, 102, 102, 103, 103, 104, 104,

In [54]:
temp = tokenized_examples.pop

In [55]:
offset_mapping = tokenized_examples.pop("offset_mapping")

In [56]:
print(offset_mapping[0])

[(0, 0), (0, 3), (3, 4), (5, 8), (9, 12), (13, 14), (14, 15), (15, 16), (17, 18), (18, 19), (19, 21), (22, 23), (23, 24), (25, 27), (28, 29), (29, 31), (31, 32), (0, 0), (0, 2), (3, 4), (4, 5), (5, 6), (7, 9), (10, 12), (13, 14), (14, 15), (15, 16), (16, 22), (23, 29), (30, 36), (36, 37), (37, 38), (39, 40), (40, 41), (41, 42), (42, 43), (44, 46), (47, 48), (48, 50), (51, 52), (52, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (63, 64), (64, 65), (65, 66), (66, 67), (68, 69), (69, 71), (71, 73), (74, 76), (76, 77), (78, 79), (80, 81), (81, 82), (83, 84), (84, 86), (87, 88), (88, 90), (90, 92), (93, 94), (94, 95), (95, 97), (98, 101), (101, 103), (104, 105), (105, 107), (107, 110), (111, 112), (112, 115), (116, 118), (118, 119), (120, 121), (121, 123), (124, 125), (125, 126), (126, 128), (128, 129), (130, 132), (132, 134), (135, 137), (137, 138), (138, 139), (140, 141), (142, 143), (143, 144), (144, 145), (145, 146), (147, 148), (148, 150), (150, 1

In [78]:
input_ids = tokenized_examples["input_ids"][0]
print(tokenized_examples["input_ids"][0])
print(type(input_ids))
print(dir(input_ids))

[101, 70672, 10622, 104431, 37650, 9966, 16605, 14646, 8880, 17730, 57952, 8854, 11018, 93222, 8932, 101958, 136, 102, 23545, 9414, 10459, 14279, 20625, 23545, 9414, 14279, 113, 10609, 10859, 22294, 114, 9632, 9543, 14279, 17730, 12030, 23545, 9637, 56356, 9414, 73295, 11903, 119, 165, 182, 165, 182, 22458, 20479, 9365, 43022, 44220, 10739, 9414, 74125, 55635, 22143, 119, 8844, 9689, 21928, 123, 45441, 9414, 74125, 73295, 9428, 52363, 16855, 10407, 45441, 9414, 74125, 78686, 8908, 103764, 11506, 119, 9644, 46216, 127, 10954, 33542, 117, 92360, 101814, 10462, 21789, 16323, 9694, 122, 120, 124, 119108, 9414, 74125, 51684, 9415, 11261, 9428, 52363, 13374, 9568, 42337, 10530, 9356, 75884, 11903, 119, 165, 182, 165, 182, 22458, 20479, 9414, 84467, 23545, 9952, 14279, 11882, 11018, 9056, 78131, 23545, 70672, 10622, 9460, 30134, 11467, 23969, 23545, 9568, 42337, 9966, 16605, 52961, 8844, 22200, 9095, 78505, 23969, 8932, 20595, 11925, 119, 9952, 73295, 9435, 40032, 11882, 8885, 17730, 10530, 1

In [76]:
print(input_ids.index(tokenizer.cls_token_id))

0


In [82]:
sequence_ids = tokenized_examples.sequence_ids(0)
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [85]:
sample_index = sample_mapping[0]

In [86]:
answers = train_dataset['answers'][sample_index]

In [87]:
print(answers)

{'answer_start': [235], 'text': ['하원']}


In [88]:
print(answers["answer_start"])

[235]


In [89]:
print(answers["text"][0])

하원


In [57]:
tokenized_examples["start_positions"] = []
tokenized_examples["end_positions"] = []

In [91]:
for i, offsets in enumerate(offset_mapping):
    # We will label impossible answers with the index of the CLS token.
    input_ids = tokenized_examples["input_ids"][i]
    cls_index = input_ids.index(tokenizer.cls_token_id)

    # Grab the sequence corresponding to that example (to know what is the context and what is the question).
    sequence_ids = tokenized_examples.sequence_ids(i)

    # One example can give several spans, this is the index of the example containing this span of text.
    sample_index = sample_mapping[i]
    answers = train_dataset["answers"][sample_index]
    # print(answers)
    # If no answers are given, set the cls_index as answer.
    if len(answers["answer_start"]) == 0:
        tokenized_examples["start_positions"].append(cls_index)
        tokenized_examples["end_positions"].append(cls_index)
    else:
        # Start/end character index of the answer in the text.
        start_char = answers["answer_start"][0]
        end_char = start_char + len(answers["text"][0])

        # Start token index of the current span in the text.
        token_start_index = 0
        while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
            token_start_index += 1

        # End token index of the current span in the text.
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
            token_end_index -= 1

        # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
        if not (
            offsets[token_start_index][0] <= start_char
            and offsets[token_end_index][1] >= end_char
        ):
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
            # Note: we could go after the last offset if the answer is the last word (edge case).
            while (
                token_start_index < len(offsets)
                and offsets[token_start_index][0] <= start_char
            ):
                token_start_index += 1
            tokenized_examples["start_positions"].append(token_start_index - 1)
            while offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            tokenized_examples["end_positions"].append(token_end_index + 1)

In [93]:
print(tokenized_examples[0])

Encoding(num_tokens=384, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [106]:
examples = {}
examples["id"] = {0,1,3,4,5}
example_id_to_index = {k: i for i, k in enumerate(examples["id"])}

In [107]:
print(example_id_to_index)

{0: 0, 1: 1, 3: 2, 4: 3, 5: 4}


In [108]:
print(mecab.morphs("안녕, 세상."))

['안녕', ',', '세상', '.']


In [123]:
import json
from collections import defaultdict

result_1 = '50.83 64.86 roberta_lstm_top10.json'
result_2 = '58.33 70.52 roberta_top10_diff.json'
result_3 = 'ensemble_dummed2.json'

with open(result_1) as fp:
    json_data_1 = json.load(fp)
with open(result_2) as fp:
    json_data_2 = json.load(fp)
with open(result_3) as fp:
    json_data_3 = json.load(fp)

ensemble = {}

for k in json_data_1.keys():
    temp = defaultdict(int)
    temp[json_data_1[k]] += 1
    temp[json_data_2[k]] += 1
    temp[json_data_3[k]] += 1
    
    i = 0
    for kk in temp.keys():
        if i < temp[kk]:
            answer = kk
            i = temp[kk]
    ensemble[k] = answer
    
with open('ensemble3.json', 'w') as fp:
    json.dump(ensemble, fp)

In [124]:
from datasets import load_from_disk

dummy = load_from_disk('/opt/ml/input/data/data/dummy_dataset')

with open('ensemble3.json') as fp:
    json_data = json.load(fp)
    
dummed = {}
for key, value in json_data.items():
    if key in dummy['train']['id']:
        print(key, value)
        idx = dummy['train']['id'].index(key)
        value = dummy['train'][idx]['answers']['text'][0]
        print('>>>', key, value)
    elif key in dummy['validation']['id']:
        print(key, value)
        idx = dummy['validation']['id'].index(key)
        value = dummy['validation'][idx]['answers']['text'][0]
        print('>>>', key, value)
    dummed[key] = value

with open('ensemble_dummed3.json', 'w') as fp:
    json.dump(dummed, fp)

mrc-1-000993 목
>>> mrc-1-000993 뒷다리
mrc-1-001662 보신 전쟁
>>> mrc-1-001662 보신 전쟁
mrc-1-000813 9월 16일
>>> mrc-1-000813 마리아나 제도에서 벗어나자
mrc-1-000715 바리사이 파
>>> mrc-1-000715 네스토리우스
mrc-1-000573 큰곰자리 운동성군
>>> mrc-1-000573 큰곰자리 운동성군
mrc-1-001735 마리아 팔코네티
>>> mrc-1-001735 마리아 팔코네티
mrc-1-000086 이승만
>>> mrc-1-000086 이승만
mrc-1-000780 꽃미남 지PD
>>> mrc-1-000780 강 노인
mrc-1-001025 특수부대「티탄즈」
>>> mrc-1-001025 특수부대「티탄즈」
mrc-1-001332 레이프 세거스탐
>>> mrc-1-001332 사카리 오라모
mrc-1-001594 자유송환
>>> mrc-1-001594 자유송환
mrc-1-000008 외투
>>> mrc-1-000008 히틀러의 외투
mrc-1-000582 녹차
>>> mrc-1-000582 소반
mrc-1-000387 미국 하원 법사위원회
>>> mrc-1-000387 전통적인 미디어들
mrc-1-001370 국제법
>>> mrc-1-001370 국제법
mrc-1-000013 요새
>>> mrc-1-000013 요새
mrc-1-001310 안티트롬빈
>>> mrc-1-001310 안티트롬빈
mrc-0-004310 일본 가도카와 쇼텐
>>> mrc-0-004310 일본 가도카와 쇼텐
mrc-1-001214 CP0의 요원
>>> mrc-1-001214 CP0
mrc-1-001689 카르무스 페르
>>> mrc-1-001689 라위스달
mrc-1-001425 260억 개
>>> mrc-1-001425 260억 개
mrc-1-000870 료스케
>>> mrc-1-000870 '스핑크스'
mrc-1-001559 소방방재센터
>>> mrc-1-001559 소방방재센터